## **This script will evaluate the membrane passive propertieis of CRH cells in the BNST between a Sapap3 KO mouse line and a WT line**
*Written by Omer Richmond* 
*Yizhar Lab*
*Weizmann institute* 

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pyabf


In [ ]:
# * I would like to plot firing rate as a function of input current* 
# * I would like to plot one example from every condition for each protocol 
# * amplitude as a function of stimulation current
# * number of EPSCs and number of IPSCs
# * Max amplitude of EPSC or IPSC in every cell
# (Show also a short sample of that raw trace)
# * Scheme of injection site 
# scheme of breeding metohd 
# image of epxression in the confocal 
# image of expression under the 1 p 
# image of cell being patched (or maybe just a scheme) 
